In [1]:
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import xgboost as xgb
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection  import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
import LightGBM as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [64]:
def adjust_param(train_data,col,i):
    y = train_data[col[i]]
    x = train_data.drop(col,axis = 1)

#     x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=2018)
#     print ("训练数据集样本数目：%d, 测试数据集样本数目：%d" % (x_train.shape[0], x_test.shape[0]))
    
    min_error = float('Inf')
    params = {}  #记录参数
    best_params = {} #记录最优的参数
    
    
    clf = lgb.LGBMRegressor(
            objective ='regression',
            learning_rate = 0.1,
            subsample = 0.8,
            colsample_bytree = 0.8,
            reg_alpha = 1,
            reg_lambda = 1,
            max_depth = 10,
            min_child_weight = 1,    
        )
    
    # 调参1 提高准确率
    print('调参1 提高准确率')
    
    for num_leaves in range(20,200,5):
        for max_depth in range(3,8,1):
            
            params['num_leaves'] = num_leaves
            params['max_depth'] = max_depth
            
            clf.set_params(**params)
            
            score = cross_val_score(clf,x,y,scoring='neg_mean_squared_log_error',cv=5)
            score = np.array(score*-1)
            print(score)
            if score.min() < min_error:
                min_error = score.min()
                best_params['num_leaves'] = num_leaves
                best_params['max_depth'] = max_depth
                
    if best_params.__contains__('num_leaves') and best_params.__contains__('max_depth'):
        print('best num_leaves:',best_params['num_leaves'])
        print('best max_depth: ',best_params['max_depth'])
        print('error:' ,min_error)
        
        clf.set_params(**best_params)
        
    
    #调参2 降低拟合
    
    print('调参2 降低过拟合')
    
    for max_bin in range(1,255,5):
        for min_data_in_leaf in range(10,200,5):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf
            
            clf.set_params(**params)
            
            score = cross_val_score(clf,x,y,scoring='neg_mean_squared_log_error',cv=5)
            score = np.array(-1*score)
            if score.min() < min_error:
                min_error = score.min()
                best_params['max_bin'] = max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf
    
    if best_params.__contains__('max_bin') and best_params.__contain__('min_data_in_leaf'):
        print('best max_bin: ',best_params['max_bin'])
        print('best min_data_in_leaf:', best_params['min_data_in_leaf'])
        print('error: ',min_error)
        clf.set_params(**best_params)
    
#     # 调参3 降低过拟合
#     print('调参3 降低过拟合')
#     for feature_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#         for bagging_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#             for bagging_freq in range(0,50,5):
#                 params['feature_fraction'] = feature_fraction
#                 params['bagging_fraction'] = bagging_fraction
#                 params['bagging_freq'] = bagging_freq
                
#                 clf.set_params(**params)
                
#                 clf.fit(x_train,y_train)
                
#                 y_pre = clf.predict(x_test)
                
#                 score = cross_val_score(clf,x,y,scoring='neg_mean_squared_log_error',cv=5)
                
#                 if score < min_error:
#                     min_error = score
#                     best_params['feature_fraction'] = feature_fraction
#                     best_params['bagging_fraction'] = bagging_fraction
#                     best_params['bagging_freq'] = bagging_freq
                    
#     if best_params.__contain__('feature_fraction'):
        
#         print('best feature_fraction:',best_params['feature_fraction'])
#         print('best bagging_fraction:',best_params['bagging_fraction'])
#         print('best bagging_freq:', best_params['bagging_freq'])
#         print('error:',min_error)
        
#         clf.set_params(**best_params)
        
    
    # 调参4 降低过拟合
    
    print("调参4：降低过拟合")
    for reg_alpha in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for reg_lambda in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
            for min_split_gain in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
                params['reg_alpha'] = reg_alpha
                params['reg_lambda'] = reg_lambda
                params['min_split_gain'] = min_split_gain
                
                clf.set_params(**params)
                
                score = cross_val_score(clf,x,y,scoring='neg_mean_squared_log_error',cv=5)
                score = np.array(-1*score)
                if score.min() < min_error:
                    min_error = score.min()
                    best_params['reg_alpha'] = reg_alpha
                    best_params['reg_lambda'] = reg_lambda
                    best_params['min_split_gain'] = min_split_gain
                   
    if best_params.__contains___('reg_alpha'):
        print('best reg_alpha: ',best_params['reg_alpha'])
        print('best reg_lambda:',best_params['reg_lambda'])
        print('error:',min_error)
        
        clf.set_params(**params)
        
    print('finished!')
    
    return clf
                
    
            
    
    
    

In [32]:
def resetIndex(df):
    vid = df['vid']
    df = df.drop('vid',axis=1)
    df = df.set_index(vid)
    return df

In [33]:
def run(train_data,test_data,col,num):
    
    train_data = resetIndex(train_data)
    
    test_data = resetIndex(test_data)
    
    x = train_data
    y = train_data[col]
    x = x.drop(col,axis=1)
    res = pd.DataFrame(index=test_data.index)
    for i in range(len(col)):
        clf = adjust_param(train_data,col,i)
        clf.fit(x,y[col[i]])
        y_pre = clf.predict(test_data)
        
        res[col[i]] = y_pre
    
    res.to_csv('./submit/'+str(num)+'.csv',encoding='gbk',header=None)
    print('finished!')
        

In [65]:
if __name__ == '__main__':
    col = ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白']
    path = './data/'
    
    p_train= path+'train_set_with_topic1.csv'
    p_test = path+'test_set_with_topic1.csv'
    
    train_data = pd.read_csv(p_train,encoding='gbk')
    test_data = pd.read_csv(p_test,encoding='gbk')
    
    run(train_data,test_data,col,20)
    
    

调参1 提高准确率
[0.01498725 0.01631951 0.0161674  0.02069518 0.01714134]
[0.01501771 0.01635576 0.01600827 0.02050765 0.01699931]
[0.01487377 0.01629624 0.01605111 0.02049062 0.01688697]
[0.01488607 0.01629072 0.01598944 0.02040695 0.01688974]
[0.01492569 0.01626694 0.01604866 0.02039248 0.01684887]


KeyboardInterrupt: 

In [62]:
a.min() < 5

True

In [ ]:
del a 